<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/Apriori_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

In [2]:
pip install apyori

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from apyori import apriori
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setMaster("local").setAppName("Market Basket with Apriori")
sc = SparkContext.getOrCreate(conf=conf)

In [5]:
file = sc.textFile('/content/drive/MyDrive/sample-datasets/store_data.csv')


In [6]:
lblitems = file.map(lambda line: line.split(','))
lblitems.count()


7501

In [7]:
wlitems = file.flatMap(lambda line: line.split(','))

In [8]:
uniqueItems = wlitems.distinct()
uniqueItems.count()

120

In [9]:
supportRdd = wlitems.map(lambda item: (item , 1))

In [10]:
supportRdd = supportRdd.reduceByKey(lambda x,y: (x+y))


In [11]:
supports = supportRdd.map(lambda item: item[1])
minSupport = supports.min()

In [12]:
supportRdd = supportRdd.filter(lambda item: item[1] >= minSupport)

In [13]:
baseRdd = supportRdd.map(lambda item: ([item[0]] , item[1]))
baseRdd.take(20)

[(['shrimp'], 536),
 (['almonds'], 153),
 (['avocado'], 250),
 (['vegetables mix'], 193),
 (['green grapes'], 68),
 (['whole weat flour'], 70),
 (['yams'], 86),
 (['cottage cheese'], 239),
 (['energy drink'], 200),
 (['tomato juice'], 228),
 (['low fat yogurt'], 574),
 (['green tea'], 991),
 (['honey'], 356),
 (['salad'], 37),
 (['mineral water'], 1788),
 (['salmon'], 319),
 (['antioxydant juice'], 67),
 (['frozen smoothie'], 475),
 (['spinach'], 53),
 (['olive oil'], 494)]

In [14]:
supportRdd = supportRdd.map(lambda item: item[0])
c2 = supportRdd.cartesian(uniqueItems)
c2 = c2.filter(lambda item: len(item) == 2)
c2.take(5)

[('shrimp', 'shrimp'),
 ('shrimp', 'almonds'),
 ('shrimp', 'avocado'),
 ('shrimp', 'vegetables mix'),
 ('shrimp', 'green grapes')]

In [15]:
l2 = c2.cartesian(lblitems)
l2 = l2.filter(lambda item: all(x in item[1] for x in item[0]))
l2.take(2)

[(('shrimp', 'shrimp'),
  ['shrimp',
   'almonds',
   'avocado',
   'vegetables mix',
   'green grapes',
   'whole weat flour',
   'yams',
   'cottage cheese',
   'energy drink',
   'tomato juice',
   'low fat yogurt',
   'green tea',
   'honey',
   'salad',
   'mineral water',
   'salmon',
   'antioxydant juice',
   'frozen smoothie',
   'spinach',
   'olive oil']),
 (('shrimp', 'shrimp'),
  ['shrimp',
   'chocolate',
   'chicken',
   'honey',
   'oil',
   'cooking oil',
   'low fat yogurt'])]

In [16]:
l2 = l2.map(lambda item: item[0])
l2 = l2.map(lambda item: (item , 1))
l2 = l2.reduceByKey(lambda x,y: (x+y))
l2 = l2.filter(lambda item: item[1] >= minSupport)
l2.take(5)

[(('shrimp', 'shrimp'), 536),
 (('shrimp', 'almonds'), 17),
 (('shrimp', 'avocado'), 23),
 (('shrimp', 'vegetables mix'), 20),
 (('shrimp', 'green grapes'), 8)]

In [17]:
supportRddL2 = baseRdd.union(l2)
supportRddL2.take(20)

[(['shrimp'], 536),
 (['almonds'], 153),
 (['avocado'], 250),
 (['vegetables mix'], 193),
 (['green grapes'], 68),
 (['whole weat flour'], 70),
 (['yams'], 86),
 (['cottage cheese'], 239),
 (['energy drink'], 200),
 (['tomato juice'], 228),
 (['low fat yogurt'], 574),
 (['green tea'], 991),
 (['honey'], 356),
 (['salad'], 37),
 (['mineral water'], 1788),
 (['salmon'], 319),
 (['antioxydant juice'], 67),
 (['frozen smoothie'], 475),
 (['spinach'], 53),
 (['olive oil'], 494)]

In [18]:
l2 = l2.map(lambda item: item[0])
supportRdd = l2
supportRdd.take(5)

[('shrimp', 'shrimp'),
 ('shrimp', 'almonds'),
 ('shrimp', 'avocado'),
 ('shrimp', 'vegetables mix'),
 ('shrimp', 'green grapes')]

In [19]:
#c3